*Homework 6: Assignment One*

Chosen Wedding Example: wedding20.py

Chosen Tools: flake8, ruff, pylint

Before tool usage: wedding20.py

In [ ]:
import itertools as it


class Wedding:
    def __init__(self):
        pass

    def panel(self, word):  # panel method (assume boundary at start) ex: abc
        if len(word) > 1:
            nminus2 = word[0]  # first letter ex: a
            nminus1 = [
                word[:2],
                word[1] + word[0],
            ]  # first 2 letter combination ex. ['ab','ba']
            nm1_calc = []
            nm2_calc = []

            for i in range(2, len(word)):  # calculate n-1 and n2 from 3rd letter and on
                nm1_calc = [x + word[i] for x in nminus1]
                nm2_calc = [x + word[i] + word[i - 1] for x in nminus2]
                nminus2 = nminus1  # new n-2 is previous n-1
                nminus1 = nm1_calc + nm2_calc  # new n-1 is newly calulated n-1 and n-2
        else:  # if it's one letter or empty word return the word itself
            nminus1 = [word]

        return nminus1  # n-1 contains final list

    def shuffle(self, word):  # method has no boundaries
        if len(word) > 2:  # example 'abcd'
            comb_stay = self.panel(
                word[1:]
            )  # assume a doesn't swap, -> a + panel('bcd')
            comb_shiftrt = self.panel(
                word[2:]
            )  # assume a swaps with b -> ba + panel('cd')
            comb_shiftlt = self.panel(
                word[1 : len(word) - 1]
            )  # assume a swaps with d -> d+panel('bc')+a
            comb_stay_form = [word[0] + i for i in comb_stay]  # adds a to combinations
            comb_shiftrt_form = [
                word[1] + word[0] + i for i in comb_shiftrt
            ]  # adds ba to the combinations
            comb_shiftlt_form = [
                word[-1] + i + word[0] for i in comb_shiftlt
            ]  # adds da based on location to the combinations
            rotate_rt = [
                word[-1] + word[: len(word) - 1]
            ]  # shift all letters one right
            rotate_lt = [word[1:] + word[0]]  # shift all letters one left
            comb_final = (
                comb_stay_form
                + comb_shiftrt_form
                + comb_shiftlt_form
                + rotate_rt
                + rotate_lt
            )  # add all combinations together
        else:  # if 2 letter word or less, return word combination using panel method
            comb_final = self.panel(word)
        return comb_final

    def barriers(self, word, block):  # methods has more than boudaries at any location
        new_word = []
        comb = []
        combf_join = []

        if (
            len(block) < 1
        ):  # if block is not provided then trat it like word with no boundaries
            comb_format = self.shuffle(word)
        else:
            for i in range(len(block)):  # example: 'A|bc|d|EF'
                if (
                    i < len(block) - 1
                ):  # separate word based on the location of the block -> new_word = ['bc','d']
                    new_word += [word[block[i] : block[i + 1]]]
                else:  # when you reach last block, include remaining letter -> new_word = ['EFA']
                    new_word += [word[block[i] :] + word[: block[0]]]
                comb = [
                    self.panel(i) for i in new_word
                ]  # perform panel on all of the new words
                # rem1 = comb
                for i in range(
                    len(comb) - 1
                ):  # multiply two word combinations together aand then multiply that with the next word combination
                    comb[i + 1] = list(
                        it.product(comb[i], comb[i + 1])
                    )  # use itertools product method to multiply each words' combinations with each other and store combination at the same location
                    comb[i + 1] = [
                        j[0] + "|" + j[1] for j in comb[i + 1]
                    ]  # for each combinations join parts together and include '|' to identify the wall; the final multiplied combina
                if (
                    0 in block
                ):  # if there is a boundary at 0th position include '|' at the start
                    comb_format = ["|" + i[: len(i) - block[0]] for i in comb[-1]]
                else:  # otherwise shift the word back to it's original position based and add '|' at teh first block location -> 'bc|d|EFA' changes to 'A|bc|d|EF'
                    comb_format = [
                        i[-1 * block[0] :] + "|" + i[: len(i) - block[0]]
                        for i in comb[-1]
                    ]
        return comb_format


def show_result(v, partial=False, ind=None):
    v.sort()
    if not partial:
        print(len(v), "\n".join(v), sep="\n")
    else:
        print(len(v), v[ind], sep="\n")


def standard_tests():
    standard = Wedding()
    res = standard.shuffle("abc")
    show_result(res)

    res = standard.shuffle("WXYZ")
    show_result(res)

    res = standard.barriers("xyz", [0])
    show_result(res)

    res = standard.shuffle("abc")
    show_result(res)

    res = standard.shuffle("abcdefXY")
    show_result(res)

    res = standard.barriers("abcDEFxyz", [2, 5, 7])
    show_result(res)

    res = standard.barriers("ABCDef", [4])
    show_result(res)

    res = standard.barriers("bgywqa", [0, 1, 2, 4, 5])
    show_result(res)

    res = standard.barriers("n", [0])
    show_result(res)
    res = standard.shuffle("hi")
    show_result(res)


def main():
    print("""Type quit to exit.
Commands:
tests
s guests
b guests n barriers
sp guests ind
bp guests n barriers ind
""")
    w = Wedding()
    while True:
        asktype = input().split()
        if not asktype or asktype[0] == "quit":
            break
        elif asktype[0] == "tests":
            standard_tests()
        elif asktype[0] == "s":
            guests = asktype[1]
            r = w.shuffle(guests)
            show_result(r)
        elif asktype[0] == "b":
            guests, nbar, bars = asktype[1], asktype[2], asktype[3:]
            r = w.barriers(guests, [int(x) for x in bars])
            show_result(r)
        elif asktype[0] == "sp":
            guests, ind = asktype[1:]
            r = w.shuffle(guests)
            show_result(r, True, int(ind))
        elif asktype[0] == "bp":
            guests, nbar, bars, ind = asktype[1], asktype[2], asktype[3:-1], asktype[-1]
            r = w.barriers(guests, [int(x) for x in bars])
            show_result(r, True, int(ind))


if __name__ == "__main__":
    main()


Flake8 Results:

In [5]:
C:\Users\hummy\Downloads\wedding20.py:4:1: E302 expected 2 blank lines, found 1
C:\Users\hummy\Downloads\wedding20.py:4:15: W291 trailing whitespace
C:\Users\hummy\Downloads\wedding20.py:7:1: W293 blank line contains whitespace
C:\Users\hummy\Downloads\wedding20.py:8:31: E262 inline comment should start with '# '
C:\Users\hummy\Downloads\wedding20.py:9:21: E225 missing whitespace around operator
C:\Users\hummy\Downloads\wedding20.py:10:33: E262 inline comment should start with '# '
C:\Users\hummy\Downloads\wedding20.py:11:80: E501 line too long (99 > 79 characters)
C:\Users\hummy\Downloads\wedding20.py:14:1: W293 blank line contains whitespace
C:\Users\hummy\Downloads\wedding20.py:15:29: E231 missing whitespace after ','
C:\Users\hummy\Downloads\wedding20.py:15:41: E261 at least two spaces before inline comment
C:\Users\hummy\Downloads\wedding20.py:15:80: E501 line too long (86 > 79 characters)
C:\Users\hummy\Downloads\wedding20.py:16:56: W291 trailing whitespace
C:\Users\hummy\Downloads\wedding20.py:19:80: E501 line too long (90 > 79 characters)
C:\Users\hummy\Downloads\wedding20.py:21:20: E225 missing whitespace around operator
C:\Users\hummy\Downloads\wedding20.py:22:1: W293 blank line contains whitespace
C:\Users\hummy\Downloads\wedding20.py:23:15: E275 missing whitespace after keyword
C:\Users\hummy\Downloads\wedding20.py:24:1: W293 blank line contains whitespace
C:\Users\hummy\Downloads\wedding20.py:26:21: E225 missing whitespace around operator
C:\Users\hummy\Downloads\wedding20.py:27:80: E501 line too long (91 > 79 characters)
C:\Users\hummy\Downloads\wedding20.py:27:92: W291 trailing whitespace
C:\Users\hummy\Downloads\wedding20.py:28:48: E261 at least two spaces before inline comment
C:\Users\hummy\Downloads\wedding20.py:28:80: E501 line too long (91 > 79 characters)
C:\Users\hummy\Downloads\wedding20.py:29:80: E501 line too long (102 > 79 characters)
C:\Users\hummy\Downloads\wedding20.py:30:80: E501 line too long (85 > 79 characters)
C:\Users\hummy\Downloads\wedding20.py:31:80: E501 line too long (104 > 79 characters)
C:\Users\hummy\Downloads\wedding20.py:32:75: E261 at least two spaces before inline comment
C:\Users\hummy\Downloads\wedding20.py:32:80: E501 line too long (122 > 79 characters)
C:\Users\hummy\Downloads\wedding20.py:33:80: E501 line too long (85 > 79 characters)
C:\Users\hummy\Downloads\wedding20.py:35:80: E501 line too long (136 > 79 characters)
C:\Users\hummy\Downloads\wedding20.py:36:80: E501 line too long (88 > 79 characters)
C:\Users\hummy\Downloads\wedding20.py:39:1: W293 blank line contains whitespace
C:\Users\hummy\Downloads\wedding20.py:40:1: W293 blank line contains whitespace
C:\Users\hummy\Downloads\wedding20.py:41:5: E303 too many blank lines (2)
C:\Users\hummy\Downloads\wedding20.py:41:37: E261 at least two spaces before inline comment
C:\Users\hummy\Downloads\wedding20.py:41:80: E501 line too long (86 > 79 characters)
C:\Users\hummy\Downloads\wedding20.py:44:9: F841 local variable 'combf_join' is assigned to but never used
C:\Users\hummy\Downloads\wedding20.py:45:1: W293 blank line contains whitespace
C:\Users\hummy\Downloads\wedding20.py:46:22: E225 missing whitespace around operator
C:\Users\hummy\Downloads\wedding20.py:46:80: E501 line too long (94 > 79 characters)
C:\Users\hummy\Downloads\wedding20.py:50:23: E225 missing whitespace around operator
C:\Users\hummy\Downloads\wedding20.py:50:80: E501 line too long (112 > 79 characters)
C:\Users\hummy\Downloads\wedding20.py:52:80: E501 line too long (100 > 79 characters)
C:\Users\hummy\Downloads\wedding20.py:54:80: E501 line too long (98 > 79 characters)
C:\Users\hummy\Downloads\wedding20.py:56:80: E501 line too long (143 > 79 characters)
C:\Users\hummy\Downloads\wedding20.py:57:56: E231 missing whitespace after ','
C:\Users\hummy\Downloads\wedding20.py:57:80: E501 line too long (196 > 79 characters)
C:\Users\hummy\Downloads\wedding20.py:58:80: E501 line too long (180 > 79 characters)
C:\Users\hummy\Downloads\wedding20.py:59:80: E501 line too long (100 > 79 characters)
C:\Users\hummy\Downloads\wedding20.py:59:101: W291 trailing whitespace
C:\Users\hummy\Downloads\wedding20.py:61:80: E501 line too long (168 > 79 characters)
C:\Users\hummy\Downloads\wedding20.py:62:80: E501 line too long (93 > 79 characters)
C:\Users\hummy\Downloads\wedding20.py:65:1: W293 blank line contains whitespace
C:\Users\hummy\Downloads\wedding20.py:66:4: E271 multiple spaces after keyword
C:\Users\hummy\Downloads\wedding20.py:66:34: E231 missing whitespace after ','
C:\Users\hummy\Downloads\wedding20.py:67:3: E111 indentation is not a multiple of 4
C:\Users\hummy\Downloads\wedding20.py:68:3: E111 indentation is not a multiple of 4
C:\Users\hummy\Downloads\wedding20.py:69:17: E231 missing whitespace after ','
C:\Users\hummy\Downloads\wedding20.py:69:30: E231 missing whitespace after ','
C:\Users\hummy\Downloads\wedding20.py:70:3: E111 indentation is not a multiple of 4
C:\Users\hummy\Downloads\wedding20.py:71:17: E231 missing whitespace after ','
C:\Users\hummy\Downloads\wedding20.py:71:24: E231 missing whitespace after ','
C:\Users\hummy\Downloads\wedding20.py:76:1: E303 too many blank lines (4)
C:\Users\hummy\Downloads\wedding20.py:77:3: E111 indentation is not a multiple of 4
C:\Users\hummy\Downloads\wedding20.py:78:3: E111 indentation is not a multiple of 4
C:\Users\hummy\Downloads\wedding20.py:79:3: E111 indentation is not a multiple of 4
C:\Users\hummy\Downloads\wedding20.py:81:3: E111 indentation is not a multiple of 4
C:\Users\hummy\Downloads\wedding20.py:82:3: E111 indentation is not a multiple of 4
C:\Users\hummy\Downloads\wedding20.py:84:3: E111 indentation is not a multiple of 4
C:\Users\hummy\Downloads\wedding20.py:85:3: E111 indentation is not a multiple of 4
C:\Users\hummy\Downloads\wedding20.py:87:3: E111 indentation is not a multiple of 4
C:\Users\hummy\Downloads\wedding20.py:88:3: E111 indentation is not a multiple of 4
C:\Users\hummy\Downloads\wedding20.py:90:3: E111 indentation is not a multiple of 4
C:\Users\hummy\Downloads\wedding20.py:91:3: E111 indentation is not a multiple of 4
C:\Users\hummy\Downloads\wedding20.py:93:3: E111 indentation is not a multiple of 4
C:\Users\hummy\Downloads\wedding20.py:94:3: E111 indentation is not a multiple of 4
C:\Users\hummy\Downloads\wedding20.py:96:3: E111 indentation is not a multiple of 4
C:\Users\hummy\Downloads\wedding20.py:97:3: E111 indentation is not a multiple of 4
C:\Users\hummy\Downloads\wedding20.py:99:3: E111 indentation is not a multiple of 4
C:\Users\hummy\Downloads\wedding20.py:100:3: E111 indentation is not a multiple of 4
C:\Users\hummy\Downloads\wedding20.py:102:3: E111 indentation is not a multiple of 4
C:\Users\hummy\Downloads\wedding20.py:103:3: E111 indentation is not a multiple of 4
C:\Users\hummy\Downloads\wedding20.py:104:3: E111 indentation is not a multiple of 4
C:\Users\hummy\Downloads\wedding20.py:105:3: E111 indentation is not a multiple of 4
C:\Users\hummy\Downloads\wedding20.py:109:1: E303 too many blank lines (3)
C:\Users\hummy\Downloads\wedding20.py:111:3: E111 indentation is not a multiple of 4
C:\Users\hummy\Downloads\wedding20.py:119:3: E111 indentation is not a multiple of 4
C:\Users\hummy\Downloads\wedding20.py:120:3: E111 indentation is not a multiple of 4
C:\Users\hummy\Downloads\wedding20.py:121:12: E225 missing whitespace around operator
C:\Users\hummy\Downloads\wedding20.py:123:7: E111 indentation is not a multiple of 4
C:\Users\hummy\Downloads\wedding20.py:123:12: E703 statement ends with a semicolon
C:\Users\hummy\Downloads\wedding20.py:123:12: E275 missing whitespace after keyword
C:\Users\hummy\Downloads\wedding20.py:125:7: E111 indentation is not a multiple of 4
C:\Users\hummy\Downloads\wedding20.py:127:7: E111 indentation is not a multiple of 4
C:\Users\hummy\Downloads\wedding20.py:128:7: E111 indentation is not a multiple of 4
C:\Users\hummy\Downloads\wedding20.py:129:7: E111 indentation is not a multiple of 4
C:\Users\hummy\Downloads\wedding20.py:129:21: E703 statement ends with a semicolon
C:\Users\hummy\Downloads\wedding20.py:131:7: E111 indentation is not a multiple of 4
C:\Users\hummy\Downloads\wedding20.py:131:13: E231 missing whitespace after ','
C:\Users\hummy\Downloads\wedding20.py:131:18: E231 missing whitespace after ','
C:\Users\hummy\Downloads\wedding20.py:131:36: E231 missing whitespace after ','
C:\Users\hummy\Downloads\wedding20.py:131:47: E231 missing whitespace after ','
C:\Users\hummy\Downloads\wedding20.py:132:7: E111 indentation is not a multiple of 4
C:\Users\hummy\Downloads\wedding20.py:133:7: E111 indentation is not a multiple of 4
C:\Users\hummy\Downloads\wedding20.py:135:7: E111 indentation is not a multiple of 4
C:\Users\hummy\Downloads\wedding20.py:135:13: E231 missing whitespace after ','
C:\Users\hummy\Downloads\wedding20.py:136:7: E111 indentation is not a multiple of 4
C:\Users\hummy\Downloads\wedding20.py:136:28: E703 statement ends with a semicolon
C:\Users\hummy\Downloads\wedding20.py:137:7: E111 indentation is not a multiple of 4
C:\Users\hummy\Downloads\wedding20.py:137:37: E703 statement ends with a semicolon
C:\Users\hummy\Downloads\wedding20.py:139:7: E111 indentation is not a multiple of 4
C:\Users\hummy\Downloads\wedding20.py:139:13: E231 missing whitespace after ','
C:\Users\hummy\Downloads\wedding20.py:139:14: F841 local variable 'nbar' is assigned to but never used
C:\Users\hummy\Downloads\wedding20.py:139:18: E231 missing whitespace after ','
C:\Users\hummy\Downloads\wedding20.py:139:23: E231 missing whitespace after ','
C:\Users\hummy\Downloads\wedding20.py:139:27: E221 multiple spaces before operator
C:\Users\hummy\Downloads\wedding20.py:139:41: E231 missing whitespace after ','
C:\Users\hummy\Downloads\wedding20.py:139:52: E231 missing whitespace after ','
C:\Users\hummy\Downloads\wedding20.py:139:66: E231 missing whitespace after ','
C:\Users\hummy\Downloads\wedding20.py:140:7: E111 indentation is not a multiple of 4
C:\Users\hummy\Downloads\wedding20.py:141:7: E111 indentation is not a multiple of 4
C:\Users\hummy\Downloads\wedding20.py:142:1: W293 blank line contains whitespace
C:\Users\hummy\Downloads\wedding20.py:145:3: E111 indentation is not a multiple of 4
C:\Users\hummy\Downloads\wedding20.py:145:9: W292 no newline at end of file

SyntaxError: unexpected character after line continuation character (458359852.py, line 1)

Ruff Results:

In [ ]:
Downloads\wedding20.py:44:9: F841 Local variable `combf_join` is assigned to but never used
   |
42 |         new_word = []
43 |         comb = []
44 |         combf_join = []
   |         ^^^^^^^^^^ F841
45 |
46 |         if len(block)<1:  # if block is not provided then trat it like word with no boundaries
   |
   = help: Remove assignment to unused variable `combf_join`

Downloads\wedding20.py:123:12: E703 [*] Statement ends with an unnecessary semicolon
    |
121 |     asktype=input().split()
122 |     if not asktype or asktype[0] == "quit":
123 |       break;
    |            ^ E703
124 |     elif asktype[0] == "tests":
125 |       standard_tests()
    |
    = help: Remove unnecessary semicolon

Downloads\wedding20.py:129:21: E703 [*] Statement ends with an unnecessary semicolon
    |
127 |       guests = asktype[1]
128 |       r = w.shuffle(guests)
129 |       show_result(r);
    |                     ^ E703
130 |     elif asktype[0] == "b":
131 |       guests,nbar,bars = asktype[1],asktype[2],asktype[3:]
    |
    = help: Remove unnecessary semicolon

Downloads\wedding20.py:136:28: E703 [*] Statement ends with an unnecessary semicolon
    |
134 |     elif asktype[0] == "sp":
135 |       guests,ind = asktype[1:]
136 |       r = w.shuffle(guests);
    |                            ^ E703
137 |       show_result(r, True, int(ind));
138 |     elif asktype[0] == "bp":
    |
    = help: Remove unnecessary semicolon

Downloads\wedding20.py:137:37: E703 [*] Statement ends with an unnecessary semicolon
    |
135 |       guests,ind = asktype[1:]
136 |       r = w.shuffle(guests);
137 |       show_result(r, True, int(ind));
    |                                     ^ E703
138 |     elif asktype[0] == "bp":
139 |       guests,nbar,bars,ind  = asktype[1],asktype[2],asktype[3:-1],asktype[-1]
    |
    = help: Remove unnecessary semicolon

Downloads\wedding20.py:139:14: F841 Local variable `nbar` is assigned to but never used
    |
137 |       show_result(r, True, int(ind));
138 |     elif asktype[0] == "bp":
139 |       guests,nbar,bars,ind  = asktype[1],asktype[2],asktype[3:-1],asktype[-1]
    |              ^^^^ F841
140 |       r = w.barriers(guests, [int(x) for x in bars])
141 |       show_result(r, True, int(ind))
    |
    = help: Remove assignment to unused variable `nbar`

Found 6 errors.
[*] 4 fixable with the `--fix` option (2 hidden fixes can be enabled with the `--unsafe-fixes` option).

Pylint:

In [ ]:
************* Module wedding20
Downloads\wedding20.py:4:14: C0303: Trailing whitespace (trailing-whitespace)
Downloads\wedding20.py:7:0: C0303: Trailing whitespace (trailing-whitespace)
Downloads\wedding20.py:14:0: C0303: Trailing whitespace (trailing-whitespace)
Downloads\wedding20.py:16:55: C0303: Trailing whitespace (trailing-whitespace)
Downloads\wedding20.py:22:0: C0303: Trailing whitespace (trailing-whitespace)
Downloads\wedding20.py:23:0: C0325: Unnecessary parens after 'return' keyword (superfluous-parens)
Downloads\wedding20.py:24:0: C0303: Trailing whitespace (trailing-whitespace)
Downloads\wedding20.py:27:91: C0303: Trailing whitespace (trailing-whitespace)
Downloads\wedding20.py:29:0: C0301: Line too long (102/100) (line-too-long)
Downloads\wedding20.py:31:0: C0301: Line too long (104/100) (line-too-long)
Downloads\wedding20.py:32:0: C0301: Line too long (122/100) (line-too-long)
Downloads\wedding20.py:35:0: C0301: Line too long (136/100) (line-too-long)
Downloads\wedding20.py:39:0: C0303: Trailing whitespace (trailing-whitespace)
Downloads\wedding20.py:40:0: C0303: Trailing whitespace (trailing-whitespace)
Downloads\wedding20.py:45:0: C0303: Trailing whitespace (trailing-whitespace)
Downloads\wedding20.py:50:0: C0301: Line too long (112/100) (line-too-long)
Downloads\wedding20.py:56:0: C0301: Line too long (143/100) (line-too-long)
Downloads\wedding20.py:57:0: C0301: Line too long (196/100) (line-too-long)
Downloads\wedding20.py:58:0: C0301: Line too long (180/100) (line-too-long)
Downloads\wedding20.py:59:100: C0303: Trailing whitespace (trailing-whitespace)
Downloads\wedding20.py:61:0: C0301: Line too long (168/100) (line-too-long)
Downloads\wedding20.py:65:0: C0303: Trailing whitespace (trailing-whitespace)
Downloads\wedding20.py:67:0: W0311: Bad indentation. Found 2 spaces, expected 4 (bad-indentation)
Downloads\wedding20.py:68:0: W0311: Bad indentation. Found 2 spaces, expected 4 (bad-indentation)
Downloads\wedding20.py:69:0: W0311: Bad indentation. Found 4 spaces, expected 8 (bad-indentation)
Downloads\wedding20.py:70:0: W0311: Bad indentation. Found 2 spaces, expected 4 (bad-indentation)
Downloads\wedding20.py:71:0: W0311: Bad indentation. Found 4 spaces, expected 8 (bad-indentation)
Downloads\wedding20.py:77:0: W0311: Bad indentation. Found 2 spaces, expected 4 (bad-indentation)
Downloads\wedding20.py:78:0: W0311: Bad indentation. Found 2 spaces, expected 4 (bad-indentation)
Downloads\wedding20.py:79:0: W0311: Bad indentation. Found 2 spaces, expected 4 (bad-indentation)
Downloads\wedding20.py:81:0: W0311: Bad indentation. Found 2 spaces, expected 4 (bad-indentation)
Downloads\wedding20.py:82:0: W0311: Bad indentation. Found 2 spaces, expected 4 (bad-indentation)
Downloads\wedding20.py:84:0: W0311: Bad indentation. Found 2 spaces, expected 4 (bad-indentation)
Downloads\wedding20.py:85:0: W0311: Bad indentation. Found 2 spaces, expected 4 (bad-indentation)
Downloads\wedding20.py:87:0: W0311: Bad indentation. Found 2 spaces, expected 4 (bad-indentation)
Downloads\wedding20.py:88:0: W0311: Bad indentation. Found 2 spaces, expected 4 (bad-indentation)
Downloads\wedding20.py:90:0: W0311: Bad indentation. Found 2 spaces, expected 4 (bad-indentation)
Downloads\wedding20.py:91:0: W0311: Bad indentation. Found 2 spaces, expected 4 (bad-indentation)
Downloads\wedding20.py:93:0: W0311: Bad indentation. Found 2 spaces, expected 4 (bad-indentation)
Downloads\wedding20.py:94:0: W0311: Bad indentation. Found 2 spaces, expected 4 (bad-indentation)
Downloads\wedding20.py:96:0: W0311: Bad indentation. Found 2 spaces, expected 4 (bad-indentation)
Downloads\wedding20.py:97:0: W0311: Bad indentation. Found 2 spaces, expected 4 (bad-indentation)
Downloads\wedding20.py:99:0: W0311: Bad indentation. Found 2 spaces, expected 4 (bad-indentation)
Downloads\wedding20.py:100:0: W0311: Bad indentation. Found 2 spaces, expected 4 (bad-indentation)
Downloads\wedding20.py:102:0: W0311: Bad indentation. Found 2 spaces, expected 4 (bad-indentation)
Downloads\wedding20.py:103:0: W0311: Bad indentation. Found 2 spaces, expected 4 (bad-indentation)
Downloads\wedding20.py:104:0: W0311: Bad indentation. Found 2 spaces, expected 4 (bad-indentation)
Downloads\wedding20.py:105:0: W0311: Bad indentation. Found 2 spaces, expected 4 (bad-indentation)
Downloads\wedding20.py:111:0: W0311: Bad indentation. Found 2 spaces, expected 4 (bad-indentation)
Downloads\wedding20.py:119:0: W0311: Bad indentation. Found 2 spaces, expected 4 (bad-indentation)
Downloads\wedding20.py:120:0: W0311: Bad indentation. Found 2 spaces, expected 4 (bad-indentation)
Downloads\wedding20.py:121:0: W0311: Bad indentation. Found 4 spaces, expected 8 (bad-indentation)
Downloads\wedding20.py:122:0: W0311: Bad indentation. Found 4 spaces, expected 8 (bad-indentation)
Downloads\wedding20.py:123:0: W0311: Bad indentation. Found 6 spaces, expected 12 (bad-indentation)
Downloads\wedding20.py:123:0: W0301: Unnecessary semicolon (unnecessary-semicolon)
Downloads\wedding20.py:124:0: W0311: Bad indentation. Found 4 spaces, expected 8 (bad-indentation)
Downloads\wedding20.py:125:0: W0311: Bad indentation. Found 6 spaces, expected 12 (bad-indentation)
Downloads\wedding20.py:126:0: W0311: Bad indentation. Found 4 spaces, expected 8 (bad-indentation)
Downloads\wedding20.py:127:0: W0311: Bad indentation. Found 6 spaces, expected 12 (bad-indentation)
Downloads\wedding20.py:128:0: W0311: Bad indentation. Found 6 spaces, expected 12 (bad-indentation)
Downloads\wedding20.py:129:0: W0311: Bad indentation. Found 6 spaces, expected 12 (bad-indentation)
Downloads\wedding20.py:129:0: W0301: Unnecessary semicolon (unnecessary-semicolon)
Downloads\wedding20.py:130:0: W0311: Bad indentation. Found 4 spaces, expected 8 (bad-indentation)
Downloads\wedding20.py:131:0: W0311: Bad indentation. Found 6 spaces, expected 12 (bad-indentation)
Downloads\wedding20.py:132:0: W0311: Bad indentation. Found 6 spaces, expected 12 (bad-indentation)
Downloads\wedding20.py:133:0: W0311: Bad indentation. Found 6 spaces, expected 12 (bad-indentation)
Downloads\wedding20.py:134:0: W0311: Bad indentation. Found 4 spaces, expected 8 (bad-indentation)
Downloads\wedding20.py:135:0: W0311: Bad indentation. Found 6 spaces, expected 12 (bad-indentation)
Downloads\wedding20.py:136:0: W0311: Bad indentation. Found 6 spaces, expected 12 (bad-indentation)
Downloads\wedding20.py:136:0: W0301: Unnecessary semicolon (unnecessary-semicolon)
Downloads\wedding20.py:137:0: W0311: Bad indentation. Found 6 spaces, expected 12 (bad-indentation)
Downloads\wedding20.py:137:0: W0301: Unnecessary semicolon (unnecessary-semicolon)
Downloads\wedding20.py:138:0: W0311: Bad indentation. Found 4 spaces, expected 8 (bad-indentation)
Downloads\wedding20.py:139:0: W0311: Bad indentation. Found 6 spaces, expected 12 (bad-indentation)
Downloads\wedding20.py:140:0: W0311: Bad indentation. Found 6 spaces, expected 12 (bad-indentation)
Downloads\wedding20.py:141:0: W0311: Bad indentation. Found 6 spaces, expected 12 (bad-indentation)
Downloads\wedding20.py:142:0: C0303: Trailing whitespace (trailing-whitespace)
Downloads\wedding20.py:145:0: C0304: Final newline missing (missing-final-newline)
Downloads\wedding20.py:145:0: W0311: Bad indentation. Found 2 spaces, expected 4 (bad-indentation)
Downloads\wedding20.py:1:0: C0114: Missing module docstring (missing-module-docstring)
Downloads\wedding20.py:4:0: C0115: Missing class docstring (missing-class-docstring)
Downloads\wedding20.py:8:4: C0116: Missing function or method docstring (missing-function-docstring)
Downloads\wedding20.py:25:4: C0116: Missing function or method docstring (missing-function-docstring)
Downloads\wedding20.py:41:4: C0116: Missing function or method docstring (missing-function-docstring)
Downloads\wedding20.py:49:12: C0200: Consider using enumerate instead of iterating with range and len (consider-using-enumerate)
Downloads\wedding20.py:44:8: W0612: Unused variable 'combf_join' (unused-variable)
Downloads\wedding20.py:66:0: C0116: Missing function or method docstring (missing-function-docstring)
Downloads\wedding20.py:76:0: C0116: Missing function or method docstring (missing-function-docstring)
Downloads\wedding20.py:109:0: C0116: Missing function or method docstring (missing-function-docstring)
Downloads\wedding20.py:122:4: R1723: Unnecessary "elif" after "break", remove the leading "el" from "elif" (no-else-break)
Downloads\wedding20.py:131:13: W0612: Unused variable 'nbar' (unused-variable)

-----------------------------------
Your code has been rated at 1.08/10

Analysis:
Both flake8 and pylint were more focused on the formatting of the code pointing out bad identation, white space issues, and whether words were too long. They also pointed out unused variables and unncessary function usage which was very interesting. The main difference between the two is that pylint provides a score at the end indicating the quality of the code based on formatting standards. This is a useful part, that pylint has over all three. Ruff was interesting as is seemed to be focused on issues that might hinder code and efficiency such as unusued variables and unnecessary colons and semicolons. 

*Assignment Two*

Now, after running the code through flake8, ruff, and pylint, we will fix the errors of the code to create a more perfect code.

Here are the results:

In [ ]:
"""
Wedding20.py assignment: Perfect Code
Using flake8, ruff, and pylint to fix formatting issues of the code
"""

# import itertools
import itertools as it


class Wedding:
    """Initalize Wedding Class"""

    def __init__(self):
        pass


    def panel(self, word):
        """panel method (assume boundary at start) ex: abc"""
        if len(word) > 1:
            nminus2 = word[0]  # first letter ex: a
            nminus1 = [
                word[:2],
                word[1] + word[0],
            ]  # first 2 letter combination ex. ['ab','ba']
            nm1_calc = []
            nm2_calc = []
            for i in range(2, len(word)):
                # calculate n-1 and n2 from 3rd letter and on
                nm1_calc = [x + word[i] for x in nminus1]
                nm2_calc = [x + word[i] + word[i - 1] for x in nminus2]
                nminus2 = nminus1  # new n-2 is previous n-1
                nminus1 = nm1_calc + nm2_calc
                # new n-1 is newly calulated n-1 and n-2
        else:  # if it's one letter or empty word return the word itself
            nminus1 = [word]
        return nminus1  # n-1 contains final list


    def shuffle(self, word):
        """method has no boundaries"""
        if len(word) > 2:  # example 'abcd'
            comb_stay = self.panel(
                word[1:]
            )  # assume a doesn't swap, -> a + panel('bcd')
            comb_shiftrt = self.panel(
                word[2:]
            )  # assume a swaps with b -> ba + panel('cd')
            comb_shiftlt = self.panel(
                word[1: len(word) - 1]
            )  # assume a swaps with d -> d+panel('bc')+a
            comb_stay_form = [word[0] + i for i in comb_stay]
            # adds a to combinations
            comb_shiftrt_form = [
                word[1] + word[0] + i for i in comb_shiftrt
            ]  # adds ba to the combinations
            comb_shiftlt_form = [
                word[-1] + i + word[0] for i in comb_shiftlt
            ]  # adds da based on location to the combinations
            rotate_rt = [
                word[-1] + word[:len(word) - 1]
            ]  # shift all letters one right
            rotate_lt = [word[1:] + word[0]]  # shift all letters one left
            comb_final = (
                comb_stay_form
                + comb_shiftrt_form
                + comb_shiftlt_form
                + rotate_rt
                + rotate_lt
            )  # add all combinations together
        else:
            # if 2 letter word or less, return word c
            # ombination using panel method
            comb_final = self.panel(word)
        return comb_final


    def barriers(self, word, block):
        """methods has more than boudaries at any location"""
        new_word = []
        comb = []
        # combf_join = []

        if (
            len(block) < 1
        ):  # if block is not provided then
            # treat it like word with no boundaries
            comb_format = self.shuffle(word)
        else:
            for i in enumerate(block):  # example: 'A|bc|d|EF'
                if (
                    i < len(block) - 1
                ):  # separate word based on the location of
                    # the block -> new_word = ['bc','d']
                    new_word += [word[block[i]: block[i + 1]]]
                else:  # when you reach last block, include remaining letter
                    # -> new_word = ['EFA']
                    new_word += [word[block[i]:] + word[: block[0]]]
                comb = [
                    self.panel(i) for i in new_word
                ]  # perform panel on all of the new words
                # rem1 = comb
                for i in range(
                    len(comb) - 1
                ):  # multiply two word combinations together and then multiply
                    # that with the next word combination
                    comb[i + 1] = list(
                        it.product(comb[i], comb[i + 1])
                    )  # use itertools product method to multiply each words'
                    # combinations with each other and store
                    # combination at the same location
                    comb[i + 1] = [
                        j[0] + "|" + j[1] for j in comb[i + 1]
                    ]  # for each combinations join parts together and include
                    # '|' to identify the wall; the final multiplied combina
                if (
                    0 in block
                ):  # if there is a boundary at
                    # 0th position include '|' at the start
                    comb_format = ["|"+i[: len(i)-block[0]] for i in comb[-1]]
                else:  # otherwise shift the word back to
                    # it's original position based
                    # and add '|' at teh first block location ->
                    # 'bc|d|EFA' changes to 'A|bc|d|EF'
                    comb_format = [
                        i[-1 * block[0], :] + "|" + i[:, len(i) - block[0]]
                        for i in comb[-1]
                    ]
        return comb_format


def show_result(v, partial=False, ind=None):
    """Function to show end of result"""
    v.sort()
    if not partial:
        print(len(v), "\n".join(v), sep="\n")
    else:
        print(len(v), v[ind], sep="\n")


def standard_tests():
    """Standard Tests to test code functionality"""
    standard = Wedding()
    res = standard.shuffle("abc")
    show_result(res)

    res = standard.shuffle("WXYZ")
    show_result(res)

    res = standard.barriers("xyz", [0])
    show_result(res)

    res = standard.shuffle("abc")
    show_result(res)

    res = standard.shuffle("abcdefXY")
    show_result(res)

    res = standard.barriers("abcDEFxyz", [2, 5, 7])
    show_result(res)

    res = standard.barriers("ABCDef", [4])
    show_result(res)

    res = standard.barriers("bgywqa", [0, 1, 2, 4, 5])
    show_result(res)

    res = standard.barriers("n", [0])
    show_result(res)
    res = standard.shuffle("hi")
    show_result(res)


def main():
    """Main Function outputs to command prompt to user interaction"""
    print("""Type quit to exit.
Commands:
tests
s guests
b guests n barriers
sp guests ind
bp guests n barriers ind
""")
    w = Wedding()
    while True:
        asktype = input().split()
        if not asktype or asktype[0] == "quit":
            pass
        elif asktype[0] == "tests":
            standard_tests()
        elif asktype[0] == "s":
            guests = asktype[1]
            r = w.shuffle(guests)
            show_result(r)
        elif asktype[0] == "b":
            guests, bars = asktype[1], asktype[3:]
            r = w.barriers(guests, [int(x) for x in bars])
            show_result(r)
        elif asktype[0] == "sp":
            guests, ind = asktype[1:]
            r = w.shuffle(guests)
            show_result(r, True, int(ind))
        elif asktype[0] == "bp":
            guests, bars, ind = (
                asktype[1],
                asktype[3:-1],
                asktype[-1],
            )
            # noqa: F841
            r = w.barriers(guests, [int(x) for x in bars])
            show_result(r, True, int(ind))
        else:
            break


if __name__ == "__main__":
    main()


Flake8 Results:

In [ ]:
PS C:\Users\hummy> flake8 "C:\Users\hummy\Downloads\wedding20.py"    
C:\Users\hummy\Downloads\wedding20.py:17:5: E303 too many blank lines (2)
C:\Users\hummy\Downloads\wedding20.py:39:5: E303 too many blank lines (2)
C:\Users\hummy\Downloads\wedding20.py:77:5: E303 too many blank lines (2)

Ruff Results:

In [ ]:
PS C:\Users\hummy> ruff check "C:\Users\hummy\Downloads\wedding20.py"
All checks passed!

Pylint Results:

In [ ]:
PS C:\Users\hummy> pylint "C:\Users\hummy\Downloads\wedding20.py"    

--------------------------------------------------------------------
Your code has been rated at 10.00/10 (previous run: 10.00/10, +0.00)

Analysis: 

After making significant changes our code score in pylint went up from 1.08/10 to 10/10 which is very significant and provides an actual metric of code improvement. For Ruff, there is a nice all checks passed marking that lets the user know there is nothing to improve. For flake8 there still is items to improve. After spending sometime, these results don't particularly make sense as to why. Each line pointed out in flake8 is a function like that doesn't have trailing whitespace or is too long. So there may be some bugginess to flake8.

The code looks to be improved, the documentation and readability of the code is much better. Some of the changes, a miniscule (such as trailing whitespace) and are hard to see while others are very noticeable (such as added docstrings). I think that pylint and flake8 provide the most comprehensive formatting help with pylint taking a the slight edge due to the score and potential bugginess mentioned earlier. Ruff is very useful, because it is well integrated with vscode and ruff can actually pick up errors in real time and offer changes.

Overall, these tools seem to improve the code in terms of readability, efficiency, and formatting. In terms of comprehensive linting and formatting the list is pylint, flake8, and ruff. In terms of integration with vscode, the list if ruff, pylint, and flake8. All of them can be used very easily, however, both ruff and pylint indicate errors in the actual codebase, where as, flake8 needs to be run separately on the file. The only caveat to everything it that your code can still have errors and not work, but be formatted and linted very well. So these tools are used, as mentioned above, for readability, efficiency and formatting but not to solve and fix bugs in the code.